In [1]:
import numpy as np
import datetime
import scipy.sparse as sp

import cplex as cp

In [2]:
def list_days_of_month(year, month):
    # Create a datetime object for the first day of the specified year and month
    date = datetime.date(year, month, 1)
    
    # Calculate the number of days in the month
    last_day_of_month = (date.replace(month=date.month % 12 + 1, day=1) - datetime.timedelta(days=1))
    num_days_in_month = last_day_of_month.day
    
    # Collect information for each day of the month
    days = []
    for day in range(1, num_days_in_month + 1):
        date = datetime.date(year, month, day)
        day_info = {
            "day": day,
            "day_of_week": date.strftime("%A"),
        }
        days.append(day_info)
    
    return days

melike 2 14 27
-        7 15 9 17 23

buğra 17
-        7 11 13 14 15

mert 1 2 3 21 22 
-        16 19 27 28

In [3]:
def mixed_integer_linear_programming(direction, A, senses, b, c, l, u, types, names):
    prob = cp.Cplex()
    
    prob.variables.add(obj= c.tolist(), lb= l.tolist(), ub= u.tolist(), types= types.tolist(), names=names.tolist())
    
    if direction == "maximize":
        prob.objective.set_sense(prob.objective.sense.maximize)
    else:
        prob.objective.set_sense(prob.objective.sense.minimize)
        
    prob.linear_constraints.add(senses= senses.tolist(), rhs= b.tolist())
    
    row_indices, col_indices= A.nonzero()
    prob.linear_constraints.set_coefficients(zip(row_indices.tolist(),
                                                col_indices.tolist(),
                                                A.data.tolist()))
    
    prob.solve()
    print(prob.write_as_string())
    
    print(prob.solution.get_status())
    print(prob.solution.status[prob.solution.get_status()])
    
    x_star= prob.solution.get_values()
    obj_star= prob.solution.get_objective_value()
    
    return (x_star, obj_star)

In [4]:
def shift_schedule_problem(input_file):  
    with open(input_file, "r") as file:
        lines = file.readlines()

    year = int(lines[0].strip())
    month = int(lines[1].strip())
    IDs = np.array(eval(lines[2].strip()))
    num_shifts = np.array(eval(lines[3].strip()))
    required_doctors = np.array(eval(lines[4].strip()))
    availabilities = np.array(eval(lines[5].strip()))

    days = list_days_of_month(year, month)

    num_doctors = np.size(IDs)
    num_days = np.size(days)
    
    names = np.array(["x_{}_{}".format(id, day['day']) for id in IDs for day in days])
    num_variables = np.size(names)

    c = availabilities.flatten()
    #önce min requiredlar G, sonra num shiftsler E, art arda günler L
    senses = np.concatenate((np.repeat("G", num_days), np.repeat("E", num_doctors), np.repeat("L", (num_days-1)*num_doctors)))
    b = np.concatenate((required_doctors, num_shifts, np.repeat(1, (num_days-1)*num_doctors)))
    l = np.repeat(0, num_variables)
    u = np.repeat(1, num_variables)
    types = np.repeat("B", num_variables)

    aij = np.repeat(1, (2*num_days*num_doctors)+(2*(num_days-1)*num_doctors))
    row = np.concatenate((np.repeat(range(num_days), num_doctors), num_days + np.repeat(range(num_doctors), num_days),
                          num_days + num_doctors + np.repeat(range((num_days-1)*num_doctors), 2)))
    
    arr = np.array([])
    for i in range(num_doctors):
       temp = (i * num_days) + np.concatenate((np.array([0]), np.repeat(range(1, num_days-1), 2), np.array([num_days-1])))
       arr = np.concatenate((arr, temp))
        
    col = np.concatenate((np.reshape(range(num_variables), (num_days, num_doctors), order = "F").reshape((num_variables,)), 
                          range(num_variables), arr)).astype(int)
    A= sp.csr_matrix((aij, (row,col)), shape= (np.size(senses), num_variables))

    x_star, obj_star = mixed_integer_linear_programming("maximize", A, senses, b, c, l, u, types, names)
    
    for i in range(num_variables):
        print(names[i] , " = " , x_star[i])
    
    return x_star

In [5]:
x_star = shift_schedule_problem('cplex.txt')

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 8 rows and 2 columns.
Reduced MIP has 104 rows, 82 columns, and 308 nonzeros.
Reduced MIP has 82 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.19 ticks)
Found incumbent of value -2.000000 after 0.06 sec. (0.51 ticks)
Probing changed sense of 39 constraints.
Probing time = 0.02 sec. (0.11 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 44 rows and 33 columns.
Aggregator did 1 substitutions.
Reduced MIP has 59 rows, 48 columns, and 172 nonzeros.
Reduced MIP has 48 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.26 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 59 rows, 48 columns, and 172 nonzeros.
Reduced MIP has 48 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.12 ticks)
Prob

Default row names c1, c2 ... being created.


\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 obj1: x_1_2 - x_1_7 - x_1_9 + x_1_14 - x_1_15 - x_1_17 - x_1_23 + x_1_27
       - x_2_7 - x_2_11 - x_2_13 - x_2_14 - x_2_15 + x_2_17 + x_3_1 + x_3_2
       + x_3_3 - x_3_16 - x_3_19 + x_3_21 + x_3_22 - x_3_27 - x_3_28
Subject To
 c1:   x_1_1 + x_2_1 + x_3_1 >= 1
 c2:   x_1_2 + x_2_2 + x_3_2 >= 2
 c3:   x_1_3 + x_2_3 + x_3_3 >= 1
 c4:   x_1_4 + x_2_4 + x_3_4 >= 0
 c5:   x_1_5 + x_2_5 + x_3_5 >= 1
 c6:   x_1_6 + x_2_6 + x_3_6 >= 1
 c7:   x_1_7 + x_2_7 + x_3_7 >= 1
 c8:   x_1_8 + x_2_8 + x_3_8 >= 1
 c9:   x_1_9 + x_2_9 + x_3_9 >= 0
 c10:  x_1_10 + x_2_10 + x_3_10 >= 1
 c11:  x_1_11 + x_2_11 + x_3_11 >= 1
 c12:  x_1_12 + x_2_12 + x_3_12 >= 2
 c13:  x_1_13 + x_2_13 + x_3_13 >= 1
 c14:  x_1_14 + x_2_14 + x_3_14 >= 2
 c15:  x_1_15 + x_2_15 + x_3_15 >= 1
 c16:  x_1_16 + x_2_16 + x_3_16 >= 1
 c17:  x_1_17 + x_2_17 + x_3_17 >= 1
 c18:  x_1_18 + x_2_18 + x_3_18 >= 1
 c19:  x_1_19 + x_2_19 + x_3_19 >= 2
 c20:  x_1_20 + x_2_20 + x_3_20 >= 1
 c21:  x_1